# Лекция 1. Задание 3

Определить порядок аппроксимации по времени и пространству на однородной сетке для следующей разностной схемы для уравнения адвекции.

$$ \frac{\partial u}{\partial t} + a \frac{\partial u}{\partial x} = 0 $$

$$ \frac{u^{n + 1}_i - u^n_i}{\Delta t} + a \frac{u^n_{i + 1} - u^n_{i - 1}}{2 \Delta x} - \frac{w}{2 \Delta x} (u^n_{i + 1} - 2 u^n_i + u^n_{i - 1}) = 0 $$

## Решение

In [1]:
import sympy as sy
from sympy import Derivative
from sympy import Eq
from sympy import Function
from sympy import IndexedBase
from sympy import O
from sympy import Symbol

In [2]:
a = Symbol('a')
t = Symbol('t')
Delta_t = Symbol('\\Delta t')
w = Symbol('w')
x = Symbol('x')
Delta_x = Symbol('\\Delta x')
i = Symbol('i', integer=True)
n = Symbol('n', integer=True)

u = Function('u')(x, t)
v = IndexedBase('v')

### Уравнение адвекции и его разностная схема

In [3]:
advection_equation = Eq(Derivative(u, t) + a * Derivative(u, x), 0)
advection_equation

Eq(a*Derivative(u(x, t), x) + Derivative(u(x, t), t), 0)

In [4]:
advection_equation_diff_scheme = Eq(
    (v[i, n + 1] - v[i, n]) / Delta_t +
    a * (v[i + 1, n] - v[i - 1, n]) / (2 * Delta_x) -
    w / (2 * Delta_x) * (v[i + 1, n] - 2 * v[i, n] + v[i - 1, n]),
    0)
advection_equation_diff_scheme

Eq(a*(v[i + 1, n] - v[i - 1, n])/(2*\Delta x) - w*(v[i + 1, n] + v[i - 1, n] - 2*v[i, n])/(2*\Delta x) + (v[i, n + 1] - v[i, n])/\Delta t, 0)

### Разложение сеточных функций $v_{i, n}$ в ряд Тейлора

In [5]:
def taylor_series(indexed, variable={Symbol('x'): Symbol('i', integer=True)},
                  diff=Symbol('\Delta x'), order=3):
    assert order > 0

    # Unpack `variable`
    var = list(variable.keys())[0]
    idx = list(variable.values())[0]

    # Get position of variable's index in `indexed`
    idx_position = 0
    for i in range(len(indexed.indices)):
        if indexed.indices[i].has(idx):
            idx_position = i

    # Set sign depending on shifted value
    sign = 1
    if indexed.indices[idx_position] == idx + 1:
        sign = 1
    elif indexed.indices[idx_position] == idx - 1:
        sign = -1

    # Set unshifted indices (e.g. [i, n])
    idxs = list(indexed.indices)
    idxs[idx_position] = idx

    # Sum up elements
    expr = indexed.base[idxs]
    for i in range(order):
        expr += (sign ** (i + 1) *
                 diff ** (i + 1) / (sy.factorial(i + 1)) *
                 Derivative(Function(indexed.base.name)(var), var, i + 1))
    expr += O(diff ** (order + 1))

    return Eq(indexed, expr)

In [6]:
v_im1_n = taylor_series(v[i - 1, n], variable={x: i}, diff=Delta_x, order=3)
v_im1_n = v_im1_n.subs(Function('v')(x), Function('v')(x, t))
v_im1_n

Eq(v[i - 1, n], v[i, n] - \Delta x*Derivative(v(x, t), x) + \Delta x**2*Derivative(v(x, t), (x, 2))/2 - \Delta x**3*Derivative(v(x, t), (x, 3))/6 + O(\Delta x**4))

In [7]:
v_ip1_n = taylor_series(v[i + 1, n], variable={x: i}, diff=Delta_x, order=3)
v_ip1_n = v_ip1_n.subs(Function('v')(x), Function('v')(x, t))
v_ip1_n

Eq(v[i + 1, n], v[i, n] + \Delta x*Derivative(v(x, t), x) + \Delta x**2*Derivative(v(x, t), (x, 2))/2 + \Delta x**3*Derivative(v(x, t), (x, 3))/6 + O(\Delta x**4))

In [8]:
v_i_np1 = taylor_series(v[i, n + 1], variable={t: n}, diff=Delta_t, order=3)
v_i_np1 = v_i_np1.subs(Function('v')(t), Function('v')(x, t))
v_i_np1

Eq(v[i, n + 1], v[i, n] + \Delta t*Derivative(v(x, t), t) + \Delta t**2*Derivative(v(x, t), (t, 2))/2 + \Delta t**3*Derivative(v(x, t), (t, 3))/6 + O(\Delta t**4))

### Выделение слагаемых с $x$ и $t$

In [9]:
advection_equation_diff_scheme_x = sum(
    advection_equation_diff_scheme.args[0].args[1:3])
advection_equation_diff_scheme_x

a*(v[i + 1, n] - v[i - 1, n])/(2*\Delta x) - w*(v[i + 1, n] + v[i - 1, n] - 2*v[i, n])/(2*\Delta x)

In [10]:
advection_equation_diff_scheme_t = (
    advection_equation_diff_scheme.args[0].args[0])
advection_equation_diff_scheme_t

(v[i, n + 1] - v[i, n])/\Delta t

### Вычисление разностей сеточных функций

In [11]:
advection_equation_diff_scheme_x_diff_1 = Eq(
    v[i + 1, n] - v[i - 1, n], v_ip1_n.args[1] - v_im1_n.args[1])
advection_equation_diff_scheme_x_diff_1

Eq(v[i + 1, n] - v[i - 1, n], 2*\Delta x*Derivative(v(x, t), x) + \Delta x**3*Derivative(v(x, t), (x, 3))/3 + O(\Delta x**4))

In [12]:
advection_equation_diff_scheme_x_diff_2 = Eq(
    v[i + 1, n] + v[i - 1, n] - 2 * v[i, n],
    v_ip1_n.args[1] + v_im1_n.args[1] - 2 * v[i, n])
advection_equation_diff_scheme_x_diff_2

Eq(v[i + 1, n] + v[i - 1, n] - 2*v[i, n], \Delta x**2*Derivative(v(x, t), (x, 2)) + O(\Delta x**4))

In [13]:
advection_equation_diff_scheme_t_diff = Eq(
    v_i_np1.args[0] - v[i, n], v_i_np1.args[1] - v[i, n])
advection_equation_diff_scheme_t_diff

Eq(v[i, n + 1] - v[i, n], \Delta t*Derivative(v(x, t), t) + \Delta t**2*Derivative(v(x, t), (t, 2))/2 + \Delta t**3*Derivative(v(x, t), (t, 3))/6 + O(\Delta t**4))

### Подстановка разностей сеточных функций в разностную схему

In [14]:
advection_equation_diff_scheme_x_approx = advection_equation_diff_scheme_x.subs({
    expr.args[0]: expr.args[1] for expr in
    [advection_equation_diff_scheme_x_diff_1,
     advection_equation_diff_scheme_x_diff_2]}).simplify()
advection_equation_diff_scheme_x_approx

a*Derivative(v(x, t), x) - \Delta x*w*Derivative(v(x, t), (x, 2))/2 + \Delta x**2*a*Derivative(v(x, t), (x, 3))/6 + O(\Delta x**3)

In [15]:
advection_equation_diff_scheme_t_approx = advection_equation_diff_scheme_t.subs({
    expr.args[0]: expr.args[1] for expr in
    [advection_equation_diff_scheme_t_diff]}).simplify()
advection_equation_diff_scheme_t_approx

Derivative(v(x, t), t) + \Delta t*Derivative(v(x, t), (t, 2))/2 + \Delta t**2*Derivative(v(x, t), (t, 3))/6 + O(\Delta t**3)

### Вычисление пределов при разностях стремящихся к нулю

In [16]:
def lim_0(expr, o_value):
    get_o = lambda arg, o_value: (O(o_value ** sy.degree(arg, gen=o_value))
                                  if (not isinstance(arg, sy.Order) and
                                      sy.degree(arg, gen=o_value) != 0)
                                  else arg)
    return sum(map(lambda arg: get_o(arg, o_value), expr.args))

In [17]:
advection_equation_diff_scheme_x_approx_lim = (
    lim_0(advection_equation_diff_scheme_x_approx, Delta_x))
advection_equation_diff_scheme_x_approx_lim

a*Derivative(v(x, t), x) + O(\Delta x)

In [18]:
advection_equation_diff_scheme_t_approx_lim = (
    lim_0(advection_equation_diff_scheme_t_approx, Delta_t))
advection_equation_diff_scheme_t_approx_lim

Derivative(v(x, t), t) + O(\Delta t)

## Результат

In [19]:
advection_equation_diff_scheme_x_approx_lim + advection_equation_diff_scheme_t_approx_lim

Derivative(v(x, t), t) + a*Derivative(v(x, t), x) + O(\Delta x) + O(\Delta t)